In [1]:
import os
import sys 
import tensorflow as tf
from dataclasses import dataclass
from pathlib import Path
from poxVisionDetection.constants import *
from poxVisionDetection.utils.common import read_yaml, create_directory, save_json
from tensorflow.keras.applications.resnet50 import preprocess_input
from poxVisionDetection import CustomException, logging
from urllib.parse import urlparse

In [6]:
# os.chdir('d:\\codes\\DeepLearning_Proj\\poxVision_detection\\research')

In [2]:
os.chdir('../')
%pwd

'd:\\codes\\DeepLearning_Proj\\poxVision_detection'

CHECKING IF THE MODEL EXISTS

In [3]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [4]:
@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model          : Path
    training_data          : Path
    all_params             : dict
    params_image_size      : list 
    params_batch_size      : int

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath   = CONFIG_FILE_PATH,
            params_filepath     = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model     = "artifacts/training/model.h5",
            training_data     = "artifacts/data_ingestion/poxVisionDataSet",
            all_params        = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )

        return eval_config

In [6]:
class Evaluation:
    def __init__(self,config : EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        valid_datagenerator         = tf.keras.preprocessing.image.ImageDataGenerator(
            preprocessing_function     = preprocess_input,
            shear_range                = 0.2,
            zoom_range                 = 0.2,
            validation_split           = 0.4,
        )

        self.valid_generator        = valid_datagenerator.flow_from_directory(
            directory                  = self.config.training_data,
            target_size                = self.config.params_image_size[:-1],
            batch_size                 = self.config.params_batch_size,
            class_mode                 = 'categorical',  
            subset                     = 'validation',
        )

    @staticmethod
    def load_model(path : Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model                   = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score                   = model.evaluate(self.valid_generator)

    def save_score(self):
        score = {'loss' : self.score[0], 'accuracy' : self.score[1]}
        save_json(path = Path('score.json'), data = score)
        

In [7]:
try:
    config         = ConfigurationManager()
    val_config     = config.get_validation_config()
    evaluation     = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    logging.exception(CustomException(e,sys))

[2023-11-13 01:23:17,961] 31 root - INFO - YAML FILE {path_to_yaml} LOADED SUCCESSFULLY 
[2023-11-13 01:23:17,963] 31 root - INFO - YAML FILE {path_to_yaml} LOADED SUCCESSFULLY 
[2023-11-13 01:23:17,965] 47 root - INFO - CREATED DIRECTORY AT : artifacts
Found 90 images belonging to 2 classes.
5/5 [==============================] - 6s 1s/step - loss: 0.7619 - accuracy: 0.5889
